# Imports

In [1]:
import pandas as pd
import numpy as np
import pickle

import pymongo

# Convert Mongo Collection into Dataframe

In [11]:
# turns mongo collection to dataframe
def getmongo(judge): # 'ginsburg'
    client = pymongo.MongoClient()
    db = client.supremes
    data = pd.DataFrame(list(db[judge].find()))
    
    data = data.dropna()
    data['text'] = data['text'].apply(lambda x: np.NaN if len(x)==0 else x)
    data = data.dropna()
    
    data['text'] = data['text'].map(lambda x: str(x).encode('ascii', 'ignore'))
    data['text'] = data['text'].map(lambda x: str(x)
                                              .replace('\\', '').replace("\'", "")
                                              .replace("b[", "").replace('b" ', '')
                                              .replace('""', '"').replace('b ', ''))
    return data

In [12]:
ginsbot = getmongo('ginsburg')

In [75]:
scaliac = getmongo('scalia')

# Merge Dataframe with 'Issues' CSV

In [19]:
# clean & merge dataframe
def merge_on_cite(dataframe):
    usCite = []
    for citation in dataframe['cite']:
        usCite.append(citation.split())
    for lists in usCite:
        if len(lists) == 5:
            del lists[-2:]
        elif len(lists ) == 4:
            del lists[-1:]
        elif len(lists) == 1:
            del lists[-1:]
    usCite = [' '.join(x) for x in usCite]
    dataframe['usCite'] = usCite
    
    df = pd.read_csv('SCDB_2016_01_justiceCentered_Vote.csv', encoding='ISO-8859-1')
    scdb = pd.concat([df['usCite'], df['issue'], df['issueArea']],
                     axis=1).drop_duplicates()
    big = dataframe.merge(scdb, how='left')
    return big


In [20]:
biggins = merge_on_cite(ginsbot)

In [80]:
bigscali = merge_on_cite(scaliac)

In [ ]:
# pickle.dump(biggins, open("biggins.p", "wb"))
# pickle.dump(bigscali, open("bigscali.p", "wb"))

# Explore Issues
Issue Areas descriptions: http://supremecourtdatabase.org/documentation.php?var=issueArea   
Issues descriptions: http://supremecourtdatabase.org/documentation.php?var=issue

In [2]:
biggins = pickle.load(open("biggins.p", "rb"))
# bigscali = pickle.load(open("bigscali.p", "rb"))

In [85]:
bigscali['issueArea'].value_counts()

1.0     66
8.0     32
2.0     30
9.0     28
3.0     20
4.0     11
10.0    11
12.0     8
5.0      6
6.0      3
7.0      2
11.0     2
13.0     1
Name: issueArea, dtype: int64

In [3]:
biggins['issueArea'].value_counts()

1.0     42
8.0     34
2.0     27
9.0     24
10.0    10
3.0      8
4.0      7
5.0      6
6.0      4
12.0     3
7.0      2
Name: issueArea, dtype: int64

## Tax

In [3]:
biggins.loc[biggins['issueArea'] == 12.0]['issue'].value_counts() 

120010.0    3
Name: issue, dtype: int64

## Affirmative Action

In [86]:
bigscali.loc[bigscali['issueArea'] == 2.0]['issue'].value_counts() 

20400.0    5
20070.0    3
20090.0    3
20040.0    3
20130.0    2
20030.0    2
20050.0    2
20210.0    2
20190.0    1
20110.0    1
20170.0    1
20020.0    1
20140.0    1
20060.0    1
20200.0    1
20160.0    1
Name: issue, dtype: int64

In [25]:
biggins.loc[biggins['issueArea'] == 2.0]['issue'].value_counts() 

20160.0    5
20070.0    3
20140.0    3
20210.0    3
20010.0    2
20150.0    2
20130.0    2
20090.0    1
20020.0    1
20050.0    1
20200.0    1
20060.0    1
20040.0    1
20320.0    1
Name: issue, dtype: int64

# Filter Issues

In [3]:
def filter_issue(dataframe, column_name, issue_list):
    data = dataframe[dataframe[column_name].isin(issue_list)]
    return data

In [10]:
value_list = [20070, 20050, 20040]
scaliaff = filter_issue(bigscali, 'issue', value_list)

In [6]:
value_list = [20040, 20070, 20050]
ginsaff = filter_issue(biggins, 'issue', value_list)

# Dataframe --> Txt File

In [14]:
def textfile(dataframe, filename):
    dataframe['text'].to_csv('%s.txt' % filename, header=False, 
                             index=False, sep=' ', mode ='w')

In [15]:
textfile(scaliaff, 'scaliaff')

In [16]:
textfile(ginstax, 'ginstax')